In [ ]:
import os
from text_summarization.constants import *
from text_summarization.utils.common import read_yaml, create_directories, get_size
import urllib.request as request
import zipfile
from text_summarization.logging import logger
from dataclasses import dataclass
from pathlib import Path

In [76]:
%pwd

'/home/sdp/python/text_summarization_project'

In [67]:
os.chdir("/home/sdp/python/text_summarization_project")

In [ ]:

@dataclass(frozen=True) # frozen=True makes the dataclass immutable, very important for configuration classes, do not miss to add it
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path 
    unzip_dir: Path

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root], verbose=True)

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir], verbose=True)

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config


class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  
        
    
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)



In [90]:

try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-07-20 21:51:32,736: INFO: common: YAML file config/config.yaml read successfully.]
[2025-07-20 21:51:32,740: INFO: common: YAML file params.yaml read successfully.]
[2025-07-20 21:51:32,742: INFO: common: Directory artifacts created successfully.]
[2025-07-20 21:51:32,743: INFO: common: Directory artifacts/data_ingestion created successfully.]
[2025-07-20 21:51:34,636: INFO: 2861295806: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 11D5:21BFCC:6B64E2:8B13BE:687DAB35
Accept-Ranges: bytes
Date: Mon, 21 Jul 2025 02:51:34 GMT
Via: 1.1 varnish
X-Served-By: cache-chi-klo

In [91]:
import os
print(os.path.exists("artifacts/dataset/samsumdata.zip"))
print(os.path.getsize("artifacts/dataset/samsumdata.zip"))

True
23627009


In [93]:
import os
print(os.path.exists("artifacts/data_ingestion/data.zip"))
print(os.path.getsize("artifacts/data_ingestion/data.zip"))

True
7903594
